In [1]:
import os, sys
sys.path.insert(1, '../../python-scripts-c6fxKDJrSsWp1xCxON1Y7g')
from api_calls import *
import ipywidgets as widgets
from ipyaggrid import Grid 
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
import time 
import json

url = "https://nomad-hzb-ce.de/nomad-oasis/api/v1"
token = os.environ['NOMAD_CLIENT_ACCESS_TOKEN']
uploads = get_all_uploads(url, token, number_of_uploads=200)
time.sleep(1)
date_picker = widgets.NaiveDatetimePicker(value=datetime.now(),disabled=False)

In [2]:
def process_upload(upload_id):
    global samples, envs, setups
    response = requests.post(f'{url}/uploads/{upload_id}/action/process', headers={'Authorization': f'Bearer {token}'})
    while True:
        time.sleep(2)
        response = requests.get(f'{url}/uploads/{upload_id}', headers={'Authorization': f'Bearer {token}'})
        if not response.json()["data"]["process_running"]:
            break
        if not (samples[samples['id'] == 'nan']['id'].any() or envs[envs['id'] == 'nan']['id'].any() or setups[setups['id'] == 'nan']['id'].any()):
            break
        with out2:
            print('processing')
    
def get_xlsx_path():
    xlsx_file_name = get_doctool_xlsx_in_upload(get_upload_id())
    if xlsx_file_name is None:
        return None
    file_path = f'../../../{get_upload_folder()}/{xlsx_file_name}'
    return file_path

def get_upload_id():
    upload_id = [u.get("upload_id") for u in uploads if upload_names.value == u.get("upload_name")]
    if len(upload_id) != 1:
        with out2:
            out2.clear_output()
            print('Could not resolve selected Upload') 
    return upload_id[0]

def get_upload_folder():
    upload_id = get_upload_id()
    for upload in os.listdir("../../.."):
        if upload_id not in upload:
            continue
        return upload

def get_doctool_in_upload(upload_id):
    query = {
        'required': {
            'data': '*'
        },
        'owner': 'visible',
        'query': {'upload_id': upload_id, 'entry_type':"CE_NOME_DocumentationTool"},
        'pagination': {
            'page_size': 150
        }
    }
    response = requests.post(
        f'{url}/entries/archive/query', headers={'Authorization': f'Bearer {token}'}, json=query)
    data = response.json()["data"]
    if not data:
        return []
    return data

def get_doctool_xlsx_in_upload(upload_id):
    data = get_doctool_in_upload(upload_id)
    if len(data) == 0:
        return None
    xlsx_file_name = data[0]['archive']['data'].get('data_file')
    return xlsx_file_name

def get_doctool_entry_id(upload_id):
    data = get_doctool_in_upload(upload_id)
    entry_id = data[0].get('entry_id')
    return entry_id

def create_empty_doctool(upload_folder, new_doc_tool_name = "nome_documentation"):
    archive =  {
            "data":{
                "m_def":"nomad_chemical_energy.schema_packages.ce_nome_package.CE_NOME_DocumentationTool",
                "name": f'{new_doc_tool_name}',
                "datetime": date_picker.value.strftime('%Y-%m-%d %H:%M:%S.%f'),
               }}
    
    import json
    archive_name = f'{new_doc_tool_name}.archive.json'
    with open(f"../../../{upload_folder}/{archive_name}", "w") as f:
        f.write(json.dumps(archive))


In [3]:
data = {}
float_columns_samples = ["active_area", "mass_coverage", "concentration","amount_relative"]
float_columns_envs = ["ph_value", "purging_temperature", "purging_time", "concentration", "amount_relative"]
def load(sheet_name):
    global samples, envs, setups, data, active
    out.clear_output()
    out2.clear_output()
    file = get_xlsx_path()
    if file is None:
        with out:
            print('This upload does not contain a valid CE_NOME_DocumentationTool. You can create a DocumentationTool in the selected upload using the "Create DocuTool" button.')
            display(widgets.VBox([button_new_docu]))
        return False
    xls = pd.ExcelFile(file)
    samples = pd.read_excel(xls, 'samples').astype({'id': 'str'})
    envs = pd.read_excel(xls, 'environments').astype({'id': 'str'})
    setups = pd.read_excel(xls, 'setups').astype({'id': 'str'})
    for i in range(10):
        samples.loc[len(samples)] = pd.Series(dtype='float64')
        envs.loc[len(envs)] = pd.Series(dtype='float64')
        setups.loc[len(setups)] = pd.Series(dtype='float64')
    grid_options = {
        'columnDefs' : [{'headerName':c,'field': c} for c in samples.columns],
        'defaultColDef': {'editable': True},
        'rowSelection': 'multiple',
        'enableRangeSelection': True,
    }
    g1 = Grid(grid_data=samples,grid_options=grid_options,sync_on_edit=True,theme='ag-theme-balham',columns_fit='auto',index=False)
    grid_options.update({'columnDefs':[{'headername':c,'field': c} for c in envs.columns]})
    g2 = Grid(grid_data=envs,grid_options=grid_options,sync_on_edit=True,theme='ag-theme-balham',columns_fit='auto',index=False)
    grid_options.update({'columnDefs':[{'headername':c,'field': c} for c in setups.columns]})
    g3 = Grid(grid_data=setups,grid_options=grid_options,sync_on_edit=True,theme='ag-theme-balham',columns_fit='auto',index=False)
    data = {'samples':g1,
            'envs':g2,
            'setups':g3
       } 
    with out:
        if active == "setups":
            display(g3)
        elif active == "envs":
            display(g2)
        else:
           display(g1)
    return True

def safe_to_float(val):
    val = str(val)
    val = val.strip()
    if val.lower() in ('', 'none', 'nan'):
        return None
    val = val.replace(',', '.')
    return val
    
def check_float_columns(df, columns):    
    for c in df.columns:
        if c == 'id':
            continue
        try:
            for c_float in columns:
                if not c.lower().startswith(c_float):
                    continue
                df.loc[:, c] = df[c].apply(safe_to_float)
                df[c] = df[c].astype('object')
                pd.set_option('future.no_silent_downcasting', True)
                df[c].astype('float64')  # not saved like this but try to catch error
        except:
            with out2:
                out2.clear_output()
                print(f'There is a float error in column {c}! Creation is aborted')
            return False
    return True

def save_table():
    global samples, envs, setups
    samples = data['samples'].grid_data_out['grid'] if  data['samples'].grid_data_out else data['samples'].grid_data_in
    samples['id'] = data['samples'].grid_data_in['id'].values
    envs = data['envs'].grid_data_out['grid'] if  data['envs'].grid_data_out else data['envs'].grid_data_in
    envs['id'] = data['envs'].grid_data_in['id'].values
    setups = data['setups'].grid_data_out['grid'] if  data['setups'].grid_data_out else data['setups'].grid_data_in
    setups['id'] = data['setups'].grid_data_in['id'].values
    check1 = check_float_columns(samples, float_columns_samples)
    check2 = check_float_columns(envs, float_columns_envs)
    if not check1 or not check2:
        return False
    file = get_xlsx_path()
    if file is None:
        with out2:
            print('This upload does not contain a valid CE_NOME_DocumentationTool.')
        return False
    with pd.ExcelWriter(file) as writer:
        samples.to_excel(writer, sheet_name='samples', index=False)
        envs.to_excel(writer, sheet_name='environments', index=False)
        setups.to_excel(writer, sheet_name='setups', index=False)
    time.sleep(0.5)
    return True

In [4]:
def on_button_clicked(b, key):
    global data, active
    active = key
    save_table()
    load(active)
def on_button_sample_clicked(b):
    on_button_clicked(b, 'samples')

def on_button_env_clicked(b):
    on_button_clicked(b, 'envs')

def on_button_setup_clicked(b):
    on_button_clicked(b, 'setups')
    
def on_new_docu_clicked(b):
    out2.clear_output()
    with out2:
        print('Creating a new CE_NOME_DocumentationTool...')
        print('This can take some time.')
    create_empty_doctool(get_upload_folder(), 'nome_documentation')
    process_upload(get_upload_id())
    load('samples')
    out2.clear_output()

def on_create_clicked(b):
    global samples, envs, setups, data, active
    saved = save_table()
    if not saved:
        return
    out2.clear_output()
    with out2:
        print('creating entries (can take some time)')
    entry_metadata = get_entry_meta_data(url, token, get_doctool_entry_id(get_upload_id()))
    set_value_in_archive(url, token, entry_metadata, 'datetime', date_picker.value.strftime('%Y-%m-%d %H:%M:%S.%f'))
    time.sleep(1.0)
    set_value_in_archive(url, token, entry_metadata, 'create_entries', True)
    
    process_upload(get_upload_id())
    with out2:
        print('entries created')
        
    time.sleep(0.5)
    load(active)
    time.sleep(0.1)
    
def on_change_uploads(change):
    global active
    active = 'samples'
    load(active)
    time.sleep(0.5)

In [5]:
out = widgets.Output()
out2 = widgets.Output()
upload_names = widgets.Dropdown(options=[u.get("upload_name","--no-name--") for u in uploads],description='Upload:')

widgets.HBox([upload_names])

In [6]:
button_sample = widgets.Button(description='Add Sample')
button_env = widgets.Button(description='Add Environment')
button_setup = widgets.Button(description='Add Setup')
button_create = widgets.Button(description='Create Entries')
button_new_docu = widgets.Button(description='Create DocuTool')
date_picker = widgets.DatePicker(value=date.today(),disabled=False)
samples = pd.DataFrame()
envs = pd.DataFrame()
setups = pd.DataFrame()

data = {}
float_columns_samples = ["active_area", "mass_coverage", "concentration","amount_relative"]
float_columns_envs = ["ph_value", "purging_temperature", "purging_time", "concentration", "amount_relative"]

active = "samples" 

button_sample.on_click(on_button_sample_clicked)
button_env.on_click(on_button_env_clicked)
button_setup.on_click(on_button_setup_clicked)
display(widgets.HBox([button_sample, button_env, button_setup, date_picker]))

button_new_docu.on_click(on_new_docu_clicked)

button_create.on_click(on_create_clicked)
upload_names.observe(on_change_uploads)

# trigger on_change_uploads once in the beginning
on_change_uploads(upload_names)

display(widgets.VBox([out, button_create, out2]))